# План работы

1. Подготовка данных
2. Обучение моделей
3. Анализ моделей
4. Выводы

## Подготовка данных

<div style="border:solid blue 2px; padding: 20px">

Импортируем библиотеки
</div>

In [1]:
import pandas as pd
import numpy as np
import time
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
warnings.filterwarnings('ignore')

<div style="border:solid blue 2px; padding: 20px">

Откроем и изучим файл
    
    
</div>

In [2]:
# чтение файла с данными и сохранение
# метод try-except поможет загрузить данные локально или глобально
try:
    data = pd.read_csv('C:/Users/autos.csv')  # Локальный путь
except:
    data = pd.read_csv('/datasets/autos.csv')  # Серверный путь

<div style="border:solid blue 2px; padding: 20px">
    
`head()` выводим на экран первые пять строк таблицы:
    </div>

In [3]:
# получение первых 5 строк таблицы
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<div style="border:solid blue 2px; padding: 20px">
    
Приведём названия в соответствие с хорошим стилем
    
</div>

<div style="border:solid blue 2px; padding: 20px">
    
Методом `str.lower()` названия столбцов приведем к нижнему регистру.
    
    
</div>

In [4]:
# перевод названий к нижнему регистру
data.columns = data.columns.str.lower()

<div style="border:solid blue 2px; padding: 20px">
    
 
Методом `describe()` получим описание данных 
    
    
</div> 

In [6]:
# описание данных
data.describe()

,price,registrationyear,power,kilometer,registrationmonth,numberofpictures,postalcode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


<div style="border:solid blue 2px; padding: 20px">
    
Методом `drop_duplicates()` удалим строки дубликаты и `reset_index(drop=True)` обновим индексацию.
    
    
</div>

In [7]:
data.duplicated().sum()

4

In [8]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
data = data.drop_duplicates().reset_index(drop=True)

<div style="border:solid blue 2px; padding: 20px">
    
 
`drop()` удалим лишние столбцы из таблицы 
    
    
</div> 

In [9]:
# удаление столбцов
data = data.drop(['datecrawled','gearbox','registrationmonth','fueltype','datecreated','numberofpictures','postalcode','lastseen'],axis = 1)

<div style="border:solid blue 2px; padding: 20px">
    

Методом `fillna()` заменим в `model` и `vehicletype` пропуски `NaN` на `other`

    
    
</div> 

In [10]:
# замена пропусков в названиях моделей автомобилей
data['model'] = data['model'].fillna('other')

In [11]:
# замена пропусков в типе автомобильного кузова
data['vehicletype'] = data['vehicletype'].fillna('other')

<div style="border:solid blue 2px; padding: 20px">
    

Методом `fillna()` заменим в `notrepaired`  пропуски `NaN` на `no_info`

    
    
</div> 

In [12]:
# замена пропусков в информации о ремонте
data['notrepaired'] = data['notrepaired'].fillna('no_info')

<div style="border:solid blue 2px; padding: 20px">

### Вывод


    
    
</div> 

<div style="border:solid blue 2px; padding: 20px">
    
**Оценка полученных данных**

Наблюдения, представленные в таблице описываются количественными и категориальными значениями.
    
Данные можно использовать для разработки приложения.
    
    
**Таблица `data` (информация об автомобилях)**:
    
`DateCrawled` — дата скачивания анкеты из базы
    
`VehicleType` — тип автомобильного кузова
    
`RegistrationYear` — год регистрации автомобиля
    
`Gearbox` — тип коробки передач
    
`Power` — мощность (л. с.)
    
`Model` — модель автомобиля
    
`Kilometer` — пробег (км)
    
`RegistrationMonth` — месяц регистрации автомобиля
    
`FuelType` — тип топлива
    
`Brand` — марка автомобиля
    
`NotRepaired` — была машина в ремонте или нет
    
`DateCreated` — дата создания анкеты
    
`NumberOfPictures` — количество фотографий автомобиля
    
`PostalCode` — почтовый индекс владельца анкеты (пользователя)
    
`LastSeen` — дата последней активности пользователя

    

**Количественный** - `Price` — цена (евро) - целевой признак
    
    

Привели названия признаков к хорошему стилю
    
Удалили строки явные дубликаты
    
Выбрали из таблицы признаки для обучения моделей

Заполнили пропуски в **Категориальных** переменных:
    
- Пропущенные значения по названиям `model` и типам `vehicletype`:

  Причина появления: человеческий фактор. 
    
  Возможно при заполнении информации об  автомобиле этот пункт не указали. 
    
  Можно сделать выбор с помощью выпадающего списка, чтобы устранить пропуски.

  Проблему решим заполнением на `other`, потому что они присутствуют в описании данных.
    
    
- Пропущенные значения в информации о ремонте `notrepaired` по аналогии заменили на `no_info`.
 
    
</div>

## Обучение моделей

<div style="border:solid blue 2px; padding: 20px">
    
 
`train_test_split()` разделим исходные данные на обучающую, валидационную и тестовую выборки
    
    
</div> 

In [13]:
# получение обучающей выборки
data_train, data_part = train_test_split(data,test_size=0.4,random_state=42)
print('Объектов обучающей выборки', data_train.shape[0])

Объектов обучающей выборки 212619


In [14]:
# получение валидационной и тестовой выборки
data_valid, data_test = train_test_split(data_part,test_size=0.5,random_state=42)
print('Объектов валидационной выборки', data_valid.shape[0])
print('Объектов тестовой выборки', data_test.shape[0])

Объектов валидационной выборки 70873
Объектов тестовой выборки 70873


<div style="border:solid blue 2px; padding: 20px">
    
 
`select_dtypes()` выделим категориальные признаки
    
    
</div> 

In [15]:
# категориальные признаки
cols_cat = data.select_dtypes(exclude='int64').columns
cat_features = list(cols_cat)
cat_features

['vehicletype', 'model', 'brand', 'notrepaired']

<div style="border:solid blue 2px; padding: 20px">
    
 
Признаки обучающей, валидационной и тестовой выборки сохраним в отдельных переменных 
    
    
- `features`
    
    
- `target` - целевой признак
    
    
</div> 

In [16]:
# сохранение признаков обучающей выбороки в отдельных переменных
features_train = data_train.drop(columns=['price'])
target_train = data_train['price']

In [17]:
# сохранение признаков валидационной выбороки в отдельных переменных
features_valid = data_valid.drop(columns=['price'])
target_valid = data_valid['price']

In [18]:
# сохранение признаков тестовой выбороки в отдельных переменных
features_test = data_test.drop(columns=['price'])
target_test = data_test['price']

<div style="border:solid blue 2px; padding: 20px">
    
   

Преобразуем категориальные признаки прямым кодированием
    
    
</div>

<div style="border:solid blue 2px; padding: 20px">
    
   

Функцией `pd.get_dummies()` с аргументом `drop_first=True` исключим попадание 
    
в ловушку фиктивных признаков
    
Посмотрим данные в случайном порядке, используя метод `sample()`
    
    
</div> 

In [19]:
# вызов функций
data_ohe = pd.get_dummies(data, drop_first=True)
data_ohe.sample(5)

,price,registrationyear,power,kilometer,vehicletype_convertible,vehicletype_coupe,vehicletype_other,vehicletype_sedan,vehicletype_small,vehicletype_suv,...,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notrepaired_no_info,notrepaired_yes
203230,4900,2009,75,80000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
257505,1000,2000,75,125000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
82842,9999,2003,275,150000,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
77276,750,1996,125,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
223006,9900,2009,0,60000,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [20]:
# признаки
features_ohe = data_ohe.drop(['price'], axis = 1)

In [21]:
# целевой признак
target_ohe = data_ohe['price']

In [22]:
# получение обучающей и тестовой выборки
features_train_ohe, features_test_ohe, target_train_ohe, target_test_ohe = train_test_split(features_ohe, target_ohe, test_size=.25, random_state=42)

<div style="border:solid blue 2px; padding: 20px">
    
 
Исследуем качество на модели — решающее дерево.

    
    
</div> 

In [23]:
tree = DecisionTreeRegressor(random_state=42)

In [24]:
parametrs_dtr = {'criterion':['mse'],'max_depth': range(1,10)}

In [25]:
%%time
time_start=time.time()
model_dtr = GridSearchCV(tree, parametrs_dtr,scoring='neg_mean_squared_error', cv=5)
model_dtr.fit(features_train_ohe, target_train_ohe)
print('Модель', model_dtr.best_estimator_)
print('Оценка', (model_dtr.best_score_ * -1)**0.5)
time_model_dtr=time.time()-time_start
print('Время выполнения ячейки кода:',time_model_dtr)

Модель DecisionTreeRegressor(max_depth=9, random_state=42)
Оценка 2183.541703577214
Время выполнения ячейки кода: 110.25816226005554
CPU times: user 1min 37s, sys: 12.9 s, total: 1min 49s
Wall time: 1min 50s


<div style="border:solid blue 2px; padding: 20px">
    
 
Исследуем качество на модели — случайный лес.
    

    
</div> 

In [26]:
forest = RandomForestRegressor(random_state=42)

In [27]:
parametrs_rfr = {'n_estimators': range (1,5),'max_depth': range (1,5)}

In [28]:
%%time
time_start=time.time()
model_rfr = GridSearchCV(forest, parametrs_rfr,scoring='neg_mean_squared_error', cv=5)
model_rfr.fit(features_train_ohe, target_train_ohe)
print('Модель', model_rfr.best_estimator_)
print('Оценка', (model_rfr.best_score_ * -1)**0.5)
time_model_rfr=time.time()-time_start
print('Время выполнения ячейки кода:',time_model_rfr)

Модель RandomForestRegressor(max_depth=4, n_estimators=4, random_state=42)
Оценка 2697.068085964894
Время выполнения ячейки кода: 167.29105305671692
CPU times: user 2min 24s, sys: 22.2 s, total: 2min 46s
Wall time: 2min 47s


<div style="border:solid blue 2px; padding: 20px">
    
 
Исследуем качество на модели — линейная регрессия.

    
    
</div> 

In [29]:
model_lr = LinearRegression()

In [30]:
parametrs_lr = {}

In [31]:
%%time
time_start=time.time()
model_lr = GridSearchCV(model_lr, parametrs_lr,scoring='neg_mean_squared_error', cv=5)
model_lr.fit(features_train_ohe,target_train_ohe)
print('Модель', model_lr)
print('Оценка', (model_lr.best_score_ * -1)**0.5)
time_model_lr=time.time()-time_start
print('Время выполнения ячейки кода:',time_model_lr)

Модель GridSearchCV(cv=5, estimator=LinearRegression(), param_grid={},
             scoring='neg_mean_squared_error')
Оценка 3324.0259080366595
Время выполнения ячейки кода: 142.9731261730194
CPU times: user 1min 35s, sys: 47.5 s, total: 2min 22s
Wall time: 2min 22s


<div style="border:solid blue 2px; padding: 20px">
    
 
Исследуем качество на модели — бустинг для категориальных признаков.

    
    
</div> 

In [32]:
params_crb = {'cat_features': cat_features,
          'random_state': 42,
          'verbose': 100}

In [33]:
model_cbr = CatBoostRegressor(**params_crb)

In [34]:
%%time
time_start=time.time()
model_cbr.fit(features_train,target_train,eval_set=(features_valid,target_valid))
time_model_cbr=time.time()-time_start
print('Время выполнения ячейки кода:',time_model_cbr)

Learning rate set to 0.118122
0:	learn: 4183.9034471	test: 4189.9311568	best: 4189.9311568 (0)	total: 398ms	remaining: 6m 37s
100:	learn: 1906.5275937	test: 1913.9662493	best: 1913.9662493 (100)	total: 17.9s	remaining: 2m 39s
200:	learn: 1837.5050575	test: 1856.3429142	best: 1856.3429142 (200)	total: 34.6s	remaining: 2m 17s
300:	learn: 1803.0374188	test: 1830.6210664	best: 1830.6210664 (300)	total: 49.6s	remaining: 1m 55s
400:	learn: 1776.8951104	test: 1813.3956210	best: 1813.3956210 (400)	total: 1m 4s	remaining: 1m 36s
500:	learn: 1756.6025320	test: 1802.0063210	best: 1802.0063210 (500)	total: 1m 20s	remaining: 1m 20s
600:	learn: 1740.6973946	test: 1794.0502778	best: 1793.9778582 (599)	total: 1m 38s	remaining: 1m 5s
700:	learn: 1728.2533555	test: 1788.7484632	best: 1788.7484632 (700)	total: 1m 54s	remaining: 48.8s
800:	learn: 1716.6318596	test: 1784.0200009	best: 1784.0200009 (800)	total: 2m 10s	remaining: 32.3s
900:	learn: 1706.2613487	test: 1779.7359963	best: 1779.7359963 (900)	tota

In [35]:
model_cbr.get_feature_importance(prettified=True)

,Feature Id,Importances
0,registrationyear,37.562130
1,power,23.044510
2,vehicletype,12.225178
3,kilometer,10.327596
4,brand,8.925390
5,model,4.710669
6,notrepaired,3.204527


<div style="border:solid blue 2px; padding: 20px">
    
 
Исследуем качество на модели — лёгкая машина градиентного бустинга.

    
    
</div> 

In [36]:
for i in cat_features:
    features_train[i] = features_train[i].astype('category')
    features_valid[i] = features_valid[i].astype('category')
    features_test[i] = features_test[i].astype('category')

In [37]:
params_lgbm = {'metric': 'rmse',
          'random_state': 42,
          'verbose': 100}

In [38]:
model_lgbm = LGBMRegressor(**params_lgbm)

In [39]:
%%time
time_start=time.time()
model_lgbm.fit(features_train,target_train, eval_set=[(features_valid, target_valid)])
time_model_lgbm=time.time()-time_start
print('Время выполнения ячейки кода:',time_model_lgbm)

[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.128987
[LightGBM] [Debug] init for col-wise cost 0.003071 seconds, init for row-wise cost 0.108626 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.100693 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 671
[LightGBM] [Info] Number of data points in the train set: 212619, number of used features: 7
[LightGBM] [Info] Start training from score 4413.603225
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 6
[1]	valid_0's rmse: 4203.86
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[2]	valid_0's rmse: 3932.21
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[3]	valid_0's rmse: 3688.05
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[4]	valid_0's rmse: 347

<div style="border:solid blue 2px; padding: 20px">

### Вывод


    
    
</div> 

<div style="border:solid blue 2px; padding: 20px">
    
 
`RSME` для моделей на валидации
    
    
    
- `DecisionTreeRegressor` - `2183.541`
    
    
- `RandomForestRegressor` - `2697.068`
    
    
- `LinearRegression` - `3324.025` 
    
    
- `CatBoostRegressor` - `1776.10`
    
    
- `LGBMRegressor` - `1796.62`

    
    
</div> 

## Анализ моделей

In [40]:
%%time
time_start=time.time()
predictions_dtr_test = model_dtr.predict(features_test_ohe) # предсказание модели DecisionTreeRegressor
time_predict_model_dtr=time.time()-time_start
rmse_dtr_test = mean_squared_error(target_test_ohe, predictions_dtr_test)**0.5
print('Время выполнения ячейки кода:',time_predict_model_dtr)
print('Средняя предсказанная стоимость :', predictions_dtr_test.mean())
print('RMSE модели DecisionTreeRegressor на тестовой выборке:', rmse_dtr_test)

Время выполнения ячейки кода: 0.21976304054260254
Средняя предсказанная стоимость : 4435.24637751906
RMSE модели DecisionTreeRegressor на тестовой выборке: 2187.7734007993563
CPU times: user 60.5 ms, sys: 92.2 ms, total: 153 ms
Wall time: 221 ms


In [42]:
%%time
time_start=time.time()
predictions_rfr_test = model_rfr.predict(features_test_ohe) # предсказание модели RandomForestRegressor
time_predict_model_rfr=time.time()-time_start               
rmse_rfr_test = mean_squared_error(target_test_ohe, predictions_rfr_test)**0.5
print('Время выполнения ячейки кода:',time_predict_model_rfr)
print('Средняя предсказанная стоимость :', predictions_rfr_test.mean())
print('RMSE модели RandomForestRegressor на тестовой выборке:', rmse_rfr_test)

Время выполнения ячейки кода: 0.15121722221374512
Средняя предсказанная стоимость : 4431.012712231336
RMSE модели RandomForestRegressor на тестовой выборке: 2689.7369026179
CPU times: user 59.9 ms, sys: 95.8 ms, total: 156 ms
Wall time: 153 ms


In [44]:
%%time
time_start=time.time()
predictions_lr_test = model_lr.predict(features_test_ohe) # предсказание модели LinearRegression
time_predict_model_lr=time.time()-time_start
rmse_lr_test = mean_squared_error(target_test_ohe, predictions_lr_test)**0.5
print('Время выполнения ячейки кода:',time_predict_model_lr)
print('Средняя предсказанная стоимость :', predictions_lr_test.mean())
print('RMSE модели LinearRegression на тестовой выборке:', rmse_lr_test)

Время выполнения ячейки кода: 0.2313709259033203
Средняя предсказанная стоимость : 4411.525934935162
RMSE модели LinearRegression на тестовой выборке: 3335.6380474607017
CPU times: user 140 ms, sys: 153 ms, total: 294 ms
Wall time: 295 ms


In [43]:
%%time
time_start=time.time()
predictions_cbr_test = model_cbr.predict(features_test) # предсказание модели CatBoostRegressor
time_predict_model_cbr=time.time()-time_start
rmse_cbr_test = mean_squared_error(target_test, predictions_cbr_test)**0.5
print('Время выполнения ячейки кода:',time_predict_model_cbr)
print('Средняя предсказанная стоимость :', predictions_cbr_test.mean())
print('RMSE модели CatBoostRegressor на тестовой выборке:', rmse_cbr_test)

Время выполнения ячейки кода: 0.5853285789489746
Средняя предсказанная стоимость : 4427.627647566589
RMSE модели CatBoostRegressor на тестовой выборке: 1782.234391120391
CPU times: user 587 ms, sys: 1.37 ms, total: 589 ms
Wall time: 587 ms


In [45]:
%%time
time_start=time.time()
predictions_lgbm_test = model_lgbm.predict(features_test) # предсказание модели LGBMRegressor
time_predict_model_lgbm=time.time()-time_start
rmse_lgbm_test = mean_squared_error(target_test, predictions_lgbm_test)**0.5
print('Время выполнения ячейки кода:',time_predict_model_lgbm)
print('Средняя предсказанная стоимость :', predictions_lgbm_test.mean())
print('RMSE модели LGBMRegressor на тестовой выборке:', rmse_lgbm_test)

Время выполнения ячейки кода: 0.6604843139648438
Средняя предсказанная стоимость : 4425.224546572326
RMSE модели LGBMRegressor на тестовой выборке: 1799.2355366929091
CPU times: user 727 ms, sys: 0 ns, total: 727 ms
Wall time: 663 ms


In [47]:
# таблица времени обучения моделей
d = {'model': ['DecisionTreeRegressor','RandomForestRegressor','LinearRegression','CatBoostRegressor','LGBMRegressor'],
     'time_model_fit': [time_model_dtr,time_model_rfr,time_model_lr,time_model_cbr,time_model_lgbm],
     'time_model_predict': [time_predict_model_dtr,time_predict_model_rfr,time_predict_model_lr,time_predict_model_cbr,time_predict_model_lgbm],
     'rmse_model': [rmse_dtr_test,rmse_rfr_test,rmse_lr_test,rmse_cbr_test,rmse_lgbm_test]
    }
df = pd.DataFrame(data=d)
df.sort_values(by='rmse_model')

,model,time_model_fit,time_model_predict,rmse_model
3,CatBoostRegressor,167.521901,0.585329,1782.234391
4,LGBMRegressor,368.736394,0.660484,1799.235537
0,DecisionTreeRegressor,110.258162,0.219763,2187.773401
1,RandomForestRegressor,167.291053,0.151217,2689.736903
2,LinearRegression,142.973126,0.231371,3335.638047


<div style="border:solid blue 2px; padding: 20px">

### Вывод


    
    
</div> 

<div style="border:solid blue 2px; padding: 20px">
    
Заказчику важны:

* качество предсказания;
* скорость предсказания;
* время обучения. 
    
`rmse_model` для модели `CatBoostRegressor` на тестовой выборке `1782.234`
    
`time_model_predict	` время предсказания для модели `CatBoostRegressor` составляет `0.585`    

`time_model_fit` время обучения для модели `CatBoostRegressor` составляет `167`

Сервис по продаже автомобилей с пробегом «Не бит, не крашен»
    
в процессе разработки приложение для привлечения новых клиентов
    
может применять модель `CatBoostRegressor`.
    
В нём можно будет быстро и объективно узнать рыночную стоимость своего автомобиля.
    

    
</div> 